In [ ]:
# Snailfish numbers, Advent of code day 18
# See https://adventofcode.com/2021/day/18 for background information. 

# Main functions

In [65]:

from math import floor, ceil 
import ast

def readSnailNo(string):

    listform = ast.literal_eval(string)
    
    return SnailNoFromList(listform)
    
    
def SnailNoFromList(listform):
    
    parta = listform[0] if type(listform[0]) == int else SnailNoFromList(listform[0]) 

    partb = listform[1] if type(listform[1]) == int else SnailNoFromList(listform[1]) 

    return SnailNo(parta,partb)
    

def add( a, b ):
    return SnailNo( a, b )
           
def depth(a):
    return 0 if type(a)==int else a.depth
            
def toprint(a):
    return str(a) if type(a)==int else a.getprint()

def splitNo(a):
    '''
        Performs a 'split' operation on a. Meant to be used iteratively.
        
        returns: (boole, SnailNo)
            boole   : T/F whether split happened
            snailno : result after first split in a occured 
    
    '''
    if type(a) == int :
        if a > 9 : 
            # We return the new entry that a should become, together with TRUE / FALSE to indicate if SPLIT happened 
            return ( True, SnailNo( floor(a/2), ceil(a/2)  ) )
        else : 
            return ( False, None )
    
    res = splitNo(a.a)
    if res[0] == True :
        a.a = res[1]
        a.setdepth()
        return (True, a )
    
    res = splitNo(a.b)
    if res[0] == True :
        a.b = res[1]
        a.setdepth()
        return (True, a )
    
    return (False, None)

def insertLeft(orig, toAdd):
    if toAdd == None     : return(orig)
    if type(orig) is int : return orig + toAdd
    else :
        orig.a = insertLeft(orig.a, toAdd)
        return orig


def insertRight(orig, toAdd):
    if toAdd == None     : return(orig)
    if type(orig) is int : return orig + toAdd
    else :
        orig.b = insertRight(orig.b, toAdd)
        return orig 

    
def reduce(snailno, d=5, verbose = False ):
    
    # performs one step of reduction 
    
    # we use a little 'hack' to see if exploding took place: simply compare the PRINTS of before and after exploding. 
    stringBefore = toprint( snailno )
    #print('Starting reduction on ', stringBefore )
    
    # Explode, and check if anything happened
    explresult = explode(snailno, d)
    stringAfter = toprint( snailno )
    
    explhappened = ( stringBefore != stringAfter )
    if explhappened : 
        if verbose : print("Exploded into ", stringAfter )
        return 1
    
    else :
        (splithappened, tmp) = splitNo(snailno)
        
        if splithappened :
            if verbose : print("Splitted into ", toprint(snailno) )
            return 2

    # If nothing happened, we return 0    
    return 0

def reduceAll( snailno, d=5, verbose = False ):
    """keep reducing until nothing happened
    decode result from decode as :
        0: nothing
        1: explode
        2: split
    
    """
    if verbose : print("Starting reduction on ", toprint(snailno))
    res = 3
    
    while res != 0 :
        res = reduce( snailno , d, verbose )
        if verbose : print("current depth: ", snailno.depth)
        
    if verbose : print("Completed reduction")
     
        
        
 
    

    

class SnailNo :
    
    def __init__(self, a, b):
        self.no = [a,b]
        self.a = a
        self.b = b
        self.setdepth()
        
    def __repr__(self):
        return toprint(self)
        
    def __print__(self):
        return toprint(self)
        
    def getprint(self):
        return '[' + toprint(self.a) + ',' + toprint(self.b) + ']'    
    
    def setdepth(self):
        self.depth = max( depth(self.a), depth(self.b) ) + 1
        
    def magnitude(self):
        ma =  self.a if type(self.a) == int else self.a.magnitude() 
        mb =  self.b if type(self.b) == int else self.b.magnitude() 
        return 3 * ma + 2 * mb
    
               
            
            

In [66]:
def explode( n, d=4 ):
    ''' explodes the leftmost snailnumber at depth d 
        Meant to be executed iteratively on lower lying numbers. 
        
        function returns (T/F, n', (L,R)) where :
            T/F  : boole whether explode actually occured
            n'   : new SnailNo after 1 explode in n
            (L,R): digits to be inserted LEFT/RIGHT of SnailNo n. 
                (If n is the topmost number, then safely discard. )
    
    '''
    # IF d larger than max depth, nothing will happen. 
    # IF d smaller than max depth, there is an error. 
    if d > n.depth : return (False, None, (None,None))
    if d < n.depth - 1 : print("Error: d = ", n ," for depth ", n.depth)
    
    
    # IF n has depth 1, then perhaps it should explode
    if depth(n) == 1 and d == 1 :
        return (True, 0, (n.a, n.b) )
    
    # Otherwise, n has depth larger than 1, we explode sub-numbers
    # Starting at a:
    if depth(n.a) == d-1 :   
        happened, newa, (l,r) = explode( n.a, d-1 )
        
        # Set new value for a
        n.a = newa
        n.setdepth()
        
        # Push value to b
        n.b = insertLeft( n.b, r )
        
        # Return success
        return(True, n, (l,None))
    
    # If nothing happened for a, then b should explode
    if depth(n.b) == d-1 :
        happened, newb, (l,r) = explode( n.b, d-1 )
        
        # Set new value for b
        n.b = newb
        n.setdepth()
        
        # Push value to a
        n.a = insertRight( n.a, l )
        
        # Return success
        return(True, n, (None, r))
    
    print("If you see this print, something's wrong: neither a nor b exploded in ", toprint(n), " at d = ", str(d) ) 
    

    
    

## Small tests

In [67]:
str1 = '[[[[4,3],4],4],[7,[[8,4],9]]]'
str2 = "[1,1]"

no1 = readSnailNo(str1)
no2 = readSnailNo(str2)
no3 = add(no1,no2)


reduceAll(no3, d=5, verbose=True)

Starting reduction on  [[[[[4,3],4],4],[7,[[8,4],9]]],[1,1]]
Exploded into  [[[[0,7],4],[7,[[8,4],9]]],[1,1]]
current depth:  5
Exploded into  [[[[0,7],4],[15,[0,13]]],[1,1]]
current depth:  4
Splitted into  [[[[0,7],4],[[7,8],[0,13]]],[1,1]]
current depth:  4
Splitted into  [[[[0,7],4],[[7,8],[0,[6,7]]]],[1,1]]
current depth:  5
Exploded into  [[[[0,7],4],[[7,8],[6,0]]],[8,1]]
current depth:  4
current depth:  4
Completed reduction


# Part 1:

In [74]:
filename = 'input18_test.txt'
file = open(filename, 'r')
list_of_inputs = []

# Create a list of clean strings, that we can interpret as SnailNo
for line in file:
    list_of_inputs.append( line.strip() )
file.close()
    
# Create list of SnailNo's
snailnos = [ readSnailNo(j) for j in list_of_inputs ]

# Iteratively add snailnos together, then reduce : 
current_number = snailnos[0]
saved_sums = []
saved_unreduced = []
for new_number in snailnos[1:]:
    print(current_number)
    print( " + " , new_number )
    current_number = add( current_number, new_number ) 
    saved_unreduced.append( toprint(current_number) )
    reduceAll( current_number, 5 )
    saved_sums.append( toprint(current_number) )
    
print("Final result: ")
print(current_number )
 
    
    
    
    

[[[4,[8,0]],[[6,9],[0,3]]],[[[2,9],[6,3]],[1,[9,9]]]]
 +  [[[2,[5,0]],[[5,6],4]],[[[2,0],[4,8]],[7,8]]]
[[[[5,6],[6,6]],[[6,7],[7,7]]],[[[7,0],[8,7]],[[7,7],[7,7]]]]
 +  [[[5,[0,6]],[9,6]],4]
[[[[6,7],[7,7]],[[7,8],[0,8]]],[[[8,8],[9,9]],[8,8]]]
 +  [[9,[5,3]],[[4,9],[8,2]]]
[[[[6,7],[7,7]],[[8,7],[7,7]]],[[[7,7],[0,8]],[[7,7],[7,6]]]]
 +  [[5,[6,2]],[[9,3],[4,[3,2]]]]
[[[[6,7],[7,8]],[[7,7],[8,8]]],[[[8,7],[7,6]],[[0,7],[6,7]]]]
 +  [[[9,[9,8]],[2,9]],[[4,[6,5]],[[6,2],9]]]
[[[[6,7],[7,7]],[[7,7],[7,7]]],[[[7,0],[7,7]],[[7,8],[8,8]]]]
 +  [[[9,8],7],[[0,[2,1]],[[9,5],[4,5]]]]
[[[[6,6],[6,6]],[[7,7],[7,8]]],[[[8,8],[7,8]],[[7,8],[0,8]]]]
 +  [[4,[0,1]],[[[3,6],5],[4,9]]]
[[[[6,7],[7,7]],[[7,7],[7,7]]],[[[6,6],[0,7]],[[7,6],[6,6]]]]
 +  [[0,9],[[3,1],3]]
[[[[7,0],[7,7]],[[8,8],[8,9]]],[[[7,8],9],[[3,1],3]]]
 +  [[[3,[7,4]],[9,8]],5]
[[[[7,6],[7,7]],[[0,7],[7,7]]],[[[7,7],[8,9]],[7,7]]]
 +  [[6,[2,8]],[0,[[6,9],[8,1]]]]
[[[[7,7],[7,6]],[[7,7],[7,6]]],[[[0,7],[7,7]],[[7,7],[8,9]]]]
 +  [[

[[[[7,7],[6,7]],[[7,7],[7,0]]],[[[7,7],[7,8]],[[8,7],[6,6]]]]
 +  [[[0,5],[3,[8,2]]],[3,2]]
[[[[7,7],[7,7]],[[7,7],[0,7]]],[[[8,8],[8,7]],[[6,7],2]]]
 +  [[[[3,3],[1,8]],[[5,8],[2,7]]],[[[1,5],9],[4,2]]]
[[[[7,7],[7,7]],[[7,8],[7,8]]],[[[7,0],[8,8]],[[8,8],[8,8]]]]
 +  [[[[7,0],5],2],[1,[[5,1],6]]]
[[[[6,6],[7,7]],[[7,8],[6,7]]],[[[7,8],[0,9]],[[8,9],[0,7]]]]
 +  [3,[[9,[9,3]],[1,[2,8]]]]
[[[[6,7],[7,7]],[[7,7],[0,7]]],[[[7,7],[7,8]],[[8,9],[6,0]]]]
 +  [[5,[[7,4],[0,3]]],[4,[[4,4],[6,8]]]]
[[[[6,6],[6,7]],[[7,8],[8,8]]],[[[8,7],[0,8]],[[7,8],[7,5]]]]
 +  [[[8,7],[5,1]],[[4,5],[7,[3,8]]]]
[[[[7,7],[7,7]],[[7,7],[7,7]]],[[[0,7],[8,8]],[[8,7],[8,8]]]]
 +  [[[[4,5],[5,5]],[[2,7],[0,5]]],[[5,[7,0]],[[9,6],5]]]
Final result: 
[[[[7,0],[7,7]],[[7,7],[7,7]]],[[[7,7],[8,7]],[[7,7],[7,6]]]]


In [77]:
print( current_number.magnitude() )

4017


# Part 2

In [93]:
import numpy as np

filename = 'input18b.txt'
file = open(filename, 'r')
list_of_inputs = []

# Create a list of clean strings, that we can interpret as SnailNo
for line in file:
    list_of_inputs.append( line.strip() )
file.close()
    
# Create list of SnailNo's
snailnos = [ readSnailNo(j) for j in list_of_inputs ]

nnums = len(list_of_inputs)

# Try adding all possible combinations together. We form a NxN matrix of magnitudes, and simply take the MAX. 
magns = np.empty((nnums, nnums), dtype = int)


for x in range(nnums):
    for y in range(nnums):
        num1 = readSnailNo( list_of_inputs[x] )
        num2 = readSnailNo( list_of_inputs[y] )
        num3 = add( num1, num2 )
        reduceAll(num3)
        magns[x,y] = num3.magnitude()



In [96]:
np.max(magns)

4583

In [97]:
magns

array([[3950, 3799, 3387, ..., 3743, 3851, 4106],
       [3737, 3661, 3207, ..., 3521, 3683, 4189],
       [3295, 2713, 2205, ..., 2471, 2735, 3247],
       ...,
       [3832, 3376, 2976, ..., 3128, 3452, 4018],
       [4144, 3754, 3102, ..., 3512, 3578, 4210],
       [4330, 3952, 3300, ..., 3782, 3776, 4504]])